## Running time
This file summarizes ways to find time and space complexity of algorithms. For the comparison, I will consider k-means implementations and walk through different ways to get the algorithm runninng time.

It's important to pay attention to these factors when implementing something, for plenty of reasons. Some of them may include environmental awareness, financial cost of memory and processors, as well as just time consumption.

In [ ]:
# Imports block
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import timeit
import time
import line_profiler
import memory_profiler 
import random

from scipy.spatial import distance
from sklearn.cluster import KMeans
from line_profiler_pycharm import profile

In [ ]:
# Import data
data1 = pd.read_csv("data_kmeans.csv", sep = ",")
data1_np = data1.to_numpy()

In [ ]:
# Here a k-means algorithm is implemented from scratch
def my_kmeans(data, clusters_num):
    """
    :param data: data to cluster
    :param clusters_num: number of clusters to separate data into
    :return: labels indentifying points belonging to a cluster, cluster centers

    """
    centers = np.ones((clusters_num, data.shape[1]))
    for i in range(clusters_num):
        centers[i] = data.sample(1)
    centers = centers[:,:2]
    data = data.to_numpy()[:,:2]
    labels = np.ones((data.shape[0],1))

    for i in range(1000):
        for j in range(clusters_num):
            dists = distance.cdist(data, centers, 'euclidean')
            label_j = np.argmin(dists, axis=1)
            inds = label_j == j
            labels[inds] = j
            center = np.mean(data[inds], axis=0)
            centers[j] = center
    return labels, centers

In [ ]:
# Function using KMeans sklearn implementation
def sklearn_kmeans(data, k):
    """
    :param data: data to cluster
    :param k: number of cluster to separate data into
    :return: labels indentifying points belonging to a cluster, cluster centers
    """
    km_alg = KMeans(n_clusters=k, init="random", random_state=70, max_iter=200)
    fit = km_alg.fit(data)
    return fit.labels_, fit.cluster_centers_

In [ ]:
labels, centers = sklearn_kmeans(data1_np,4)
plt.scatter(data1_np[:, 0], data1_np[:, 1], c=labels)

In [ ]:
labels1, centers1 = my_kmeans(data1, 4)
plt.scatter(data1_np[:, 0], data1_np[:, 1], c=labels1)

Firstly, time the two algorithms by their running time for comparison.

In [ ]:
start_time = time.time()

my_kmeans(data1, 4)

stop_time = time.time()

print("This took %s seconds to run" % (stop_time-start_time))

In [ ]:
start_time = time.time()

sklearn_kmeans(data1_np, 4)

stop_time = time.time()

print("This took %s seconds to run" % (stop_time-start_time))

Time module measures run time using system time and while it's somewhat representative, there can be small changes in run time, and in `timeit`, we run the code several times to find the average run time.

In [ ]:
%%timeit

my_kmeans(data1, 4)

In [ ]:
%%timeit

sklearn_kmeans(data1_np, 4)

Obviously, my implementation of k-means is slower than the one from sklearn. 
When we _benchmark_ code, we examine how fast each piece of the code is. 

To look into the runtime in more detail, we profile code using line-by-line profiler.
Benchmarking as a whole allows us to determine if we need to edit or change any lines due to _bottlenecking_

### For a dictionary reminder:
#### Epoch
An epoch is the point where the algorithm has seen the 
whole set of training data exactly one time. While the input data may be of a different size, it is all same for epoch. Therefore the epochs are convinient for comparing algorithms, when the data they're applied to differs by size.

#### Batch Size

_Batch size_ is the number of training examples that the algorithm "sees" in each pass.

#### Iterations
_Iterations_ is the number of iterations needed to complete one epoch for the algorithm.

In [ ]:
%prun sklearn_kmeans(data1_np, 4)

In [ ]:
%prun my_kmeans(data1, 4)

From looking into profiler, we can determine which parts of code are called most frequently, how much time each call takes, and how much they total into

Most "expensive" calls we have in sklearn_kmeans are euclidian distances function, validation and object reductioon

For my implementation, some of the functions that take more time are distance, means, reduction. Because it's iterating a constant number of times, the code runs for 800 epochs, hence a bigger number.

In [ ]:
%load_ext line_profiler

While %prune looks a little intimidating to analyze, line profiler provides general statistics for each line in the file separately. It's convinient for knowing what should be changed to improve efficien

In [ ]:
%lprun -f my_kmeans my_kmeans(data1, 4)

In [ ]:
%lprun -f sklearn_kmeans sklearn_kmeans(data1_np, 4)

In some cases it's useful to know how much memory an algorithm takes while running. It's possible to explore using memory profiler. From the memory standpoint, my algorithns is not that bad :)

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit my_kmeans(data1, 4)

In [ ]:
%memit sklearn_kmeans(data1_np, 4)